Script for creating and training the LSTM network models used to generate classical music.

Matilda Wikström

Jesper Larsson

Links to good stuff!

https://github.com/craffel/pretty-midi

https://towardsdatascience.com/generate-piano-instrumental-music-by-using-deep-learning-80ac35cdbd2e





In [1]:
#Imports
import torch
import pretty_midi
import numpy as np
import unicodedata
import re
import numpy as np
from random import shuffle, seed
import os
import io
import time
from tqdm import tnrange, tqdm_notebook, tqdm
# "Our code"
import preprocessing as pp
#import generatemidi as gm # Broken file, fix
import tokenizer as token

In [2]:
# Using pretty_midi example
fs=30
dict_note = {}
midi_pretty_format = pretty_midi.PrettyMIDI('test.midi')
piano_midi = midi_pretty_format.instruments[0] # Get the piano channels
piano_roll = piano_midi.get_piano_roll(fs=fs)
dict_note[0] = piano_roll

print(len(piano_roll[0]))

29070


In [ ]:
#Download the required data create folder structure
#Make sure we are in the correct folder

assert (os.path.basename(os.getcwd())=='LSTM-MusicGenerator'), "Wrong working dir"

#Download the MAESTRO Dataset
if not os.path.isfile('./maestro-v1.0.0-midi.zip'):
    !wget https://storage.googleapis.com/magentadata/datasets/maestro/v1.0.0/maestro-v1.0.0-midi.zip

#Check if we have extracted the files
if not os.path.isdir('./maestro-v1.0.0-midi'):
    !unzip maestro-v1.0.0-midi.zip


In [3]:
# Preprocessing
list_all_midi = pp.get_list_midi() # 
sampled_200_midi = list_all_midi[0:100]
batch = 1
start_index = 0
note_tokenizer = token.NoteTokenizer()
import pretty_midi

for i in tqdm_notebook(range(len(sampled_200_midi))):
    dict_time_notes = token.generate_dict_time_notes(sampled_200_midi, batch_song=1, start_index=i, use_tqdm=False, fs=5)
    full_notes = token.process_notes_in_song(dict_time_notes)
    for note in full_notes:
        note_tokenizer.partial_fit(list(note.values()))
note_tokenizer.add_new_note('e') # Add empty notes



In [7]:
#Hyper parameters
seq_len = 50
EPOCHS = 4
BATCH_SONG = 16
BATCH_NNET_SIZE = 96
TOTAL_SONGS = len(sampled_200_midi)
FRAME_PER_SECOND = 5


unique_notes = note_tokenizer.unique_word #Used in our output layer to map
#unique_notes+1

In [10]:
# Model
import torch.nn as nn
import torch.nn.functional as F

#Input dimension = 50
#Batch dimension = ? May be one, since we are putting them on 
#Hidden size, try different versions
   
input_size = 50
hidden_size = 20
num_layers = 1
is_bidirectional = True
#dropout_rate = 0.75
dropout_rate = 0
batch_size = 1

if is_bidirectional:
    num_directions = 2
else:
    num_directions = 1

input = torch.randn(1, batch_size, input_size)
h0 = torch.randn(num_layers*num_directions, batch_size, hidden_size)
c0 = torch.randn(num_layers*num_directions, batch_size, hidden_size)



class our_LSTM(torch.nn.Module):
    def __init__(self, h0, c0 ):
        super().__init__()
        self.h0 = h0
        self.c0 = c0
        
        self.lstm = torch.nn.LSTM(input_size, hidden_size, num_layers, dropout=dropout_rate, bidirectional=is_bidirectional)
        self.fc = torch.nn.Linear(num_directions * hidden_size, num_directions * hidden_size)
        self.softmax = torch.nn.Softmax(dim=-1)
        
    def forward(self, inp):
        out, (hn, cn) = self.lstm(inp, (self.h0, self.c0))
        out = self.fc(out)
        out = self.softmax(out)
        
        return out, (hn,cn)
    


In [27]:
seq_len = 50
EPOCHS = 4
BATCH_SONG = 16
BATCH_NNET_SIZE = 96
TOTAL_SONGS = len(sampled_200_midi)
FRAME_PER_SECOND = 5

batch_song = BATCH_SONG
frame_per_second = FRAME_PER_SECOND
inputs_nnet_large, outputs_nnet_large = token.generate_batch_song(
            sampled_200_midi, batch_song, start_index=i, fs=frame_per_second, 
            seq_len=seq_len, use_tqdm=False)
print(inputs_nnet_large[0]) #For each timestep we get a set of notes. If the next timestep contains the same note, it is held longer
# LOL
print(outputs_nnet_large[0])

trans_in = note_tokenizer.transform(inputs_nnet_large)
trans_out = note_tokenizer.transform(outputs_nnet_large)

print((trans_in.shape))
print((trans_out.shape))

['e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', '62,65']
['61,63,65,67']
(2085, 50)
(2085, 1)


In [20]:
## Training loop

model = our_LSTM(h0,c0)

model(trans_in)







TypeError: 'int' object is not callable